In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.sparse as sparse

In [2]:
def open_lp(file):
    with open(file) as f:
        lines = f.readlines()

    lines = np.array(lines)

    objective_start = np.argwhere(lines == 'Minimize\n')[0, 0]
    constraints_start = np.argwhere(lines == 'Subject To\n')[0, 0]
    bounds_start = np.argwhere(lines == 'Bounds\n')[0, 0]
    end = np.argwhere(lines == 'End\n')[0, 0] 
    print(bounds_start)
    return (
        lines[objective_start+1:constraints_start], 
        lines[constraints_start+1:bounds_start], 
        lines[bounds_start+1:end]
    )

def open_mps(file):
    
    with open(file) as f:
        lines = f.readlines()
    
    lines = np.array(lines)
    row_start = np.argwhere(lines == 'ROWS\n')[0, 0]
    column_start = np.argwhere(lines == 'COLUMNS\n')[0, 0]
    rhs_start = np.argwhere(lines == 'RHS\n')[0, 0]
    bound_start = np.argwhere(lines == 'BOUNDS\n')[0, 0]
    
    return (
        lines[row_start+1:column_start], 
        lines[column_start+1:rhs_start], 
        lines[rhs_start+1:bound_start], 
        lines[bound_start+1:-1]
    )

In [3]:
def read_variables(columns_section, bounds_section):
    
    def read_bounds(bound_type, value, entry):
        if bound_type == 'UP':
            return (entry[0], entry[1], float(value))
        elif bound_type == 'FX':
            return (entry[0], float(value), float(value))
        elif bound_type == 'LO':
            return (entry[0], float(value), entry[2])
        else:
            print(f'Bound type {bound_type} is not supported.')
    
    variables = {}
    # keep int_marker, i.e., the information whether a variable is discrete in here. Not necessary 
    # immidiately but allows easy expansion of the code. 
    int_marker = False
    
    for line in columns_section:
        if not "'MARKER'" in line:
            variables[line.lstrip().split(' ')[0]] = (int_marker, None, None)
        else:
            int_marker = "'INTORG'" in line
            
    for line in bounds_section:
        try:
            bound_type, _, variable_name, value = line.split()
            variables[variable_name] = read_bounds(bound_type, value, variables[variable_name])
        except:
            print(line)
            break
        
    variable_indices = {name: index for index, (name, _) in enumerate(variables.items())}
    bounds, discrete = [None for i in range(len(variables))], [None for i in range(len(variables))]
    for name, entry in variables.items():
        bounds[variable_indices[name]] = entry[1:]
        discrete[variable_indices[name]] = entry[0]
    
    return variable_indices, bounds, discrete

In [68]:
def read_equalities(rows_section, columns_section, rhs_section, variable_indices):
    
    def vectorize(rows_lhs):
        row_indices = {name: index for index, name in enumerate(rows_lhs.keys())}
        entries, index_1, index_2 = [], [], []
        for row, terms in rows_lhs.items():
            for entry, variable in terms:
                entries.append(entry)
                index_1.append(row_indices[row])
                index_2.append(variable_indices[variable])
        
        matrix_lhs = sparse.coo_array(
            (entries, (index_1, index_2)),
            shape=(len(row_indices), len(variable_indices))
        )
        
        vector_rhs = np.zeros(len(rows_lhs))
        for name, index in row_indices.items():
            try:
                vector_rhs[index] = rows_rhs[name]
            except:
                None
        
        return matrix_lhs, vector_rhs
        
    
    rows_lhs = {
        line.split()[1]: []
        for line in rows_section
    }
    
    for term in columns_section:
        if not "'MARKER'" in term:
            column, row, entry = term.split()
            rows_lhs[row].append([float(entry), column])
    
    rows_rel = {
        line.split()[1]: line.split()[0]
        for line in rows_section
    }
    
    rows_rhs = {
        line.split()[1]: float(line.split()[2])
        for line in rhs_section
    }
    
    # construct upper bound and equality constraint matrices
    ubs = {key: entry for key, entry in rows_lhs.items() if rows_rel[key] == 'L'}
    lbs = {key: entry for key, entry in rows_lhs.items() if rows_rel[key] == 'G'}
    eqs = {key: entry for key, entry in rows_lhs.items() if rows_rel[key] == 'E'}
    obj = {key: entry for key, entry in rows_lhs.items() if rows_rel[key] == 'N'}
    assert len(obj) == 1, 'There has to be exactly one objective!'
    
    A_ub, b_ub = vectorize(ubs)#sparse.vstack([build_mat(ubs), -build_mat(lbs)])
    A_lb, b_lb = vectorize(lbs)
    # solver convention: only use upper bounds -> translate lower bounds
    A_ub, b_ub = sparse.vstack([A_ub, -A_lb]), np.stack([b_ub, -b_lb])
    # keep going with equations and the objective
    A_eq, b_eq = vectorize(eqs)
    c, offset = vectorize(obj)
    # the objective is usually not sparse
    c = -c.todense()
    offset = offset[0]
    
    return c, A_ub, b_ub, A_eq, b_eq, offset

In [69]:
file = 'MIRPs/test_instance.mps'

rows_section, columns_section, rhs_section, bounds_section = open_mps(file)
variable_indices, bounds, discrete = read_variables(columns_section, bounds_section)
c, A_ub, b_ub, A_eq, b_eq, offset = read_equalities(rows_section, columns_section, rhs_section, variable_indices)

In [70]:
opt.linprog(c, A_ub, b_ub, A_eq, b_eq, bounds)

           con: array([0.])
 crossover_nit: 0
         eqlin:  marginals: array([-9.])
  residual: array([0.])
           fun: -80.0
       ineqlin:  marginals: array([-0., -0.])
  residual: array([0., 2.])
         lower:  marginals: array([0., 0., 0.])
  residual: array([inf,  2., inf])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 0
         slack: array([0., 2.])
        status: 0
       success: True
         upper:  marginals: array([ -1., -13.,   0.])
  residual: array([ 0.,  0., inf])
             x: array([4., 1., 8.])

In [30]:
variable_indices

{'XONE': 0, 'YTWO': 1, 'ZTHREE': 2}

In [7]:
bounds

[(None, 4.0), (-1.0, 1.0), (None, None)]

In [8]:
discrete

[False, False, False]

In [9]:
c = np.array([-1, 0, -1])

A_ub = np.array([[1, 1, 1]])
test = sparse.coo_array(
    (
        [1, 1], 
        ([0, 0], [0, 2])
    ), 
    shape=(1, 3)
)
b_ub = 2

opt.linprog(c, test, b_ub, bounds=[(0,1), (0,1), (0,1)])

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -2.0
       ineqlin:  marginals: array([-0.])
  residual: array([0.])
         lower:  marginals: array([0., 0., 0.])
  residual: array([1., 0., 1.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 0
         slack: array([0.])
        status: 0
       success: True
         upper:  marginals: array([-1.,  0., -1.])
  residual: array([0., 1., 0.])
             x: array([1., 0., 1.])

In [10]:
test.todense()

array([[1, 0, 1]])